<a href="https://colab.research.google.com/github/Samiul1403001/Python_projects/blob/master/Classifier_4_class_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Connecting with google drive

from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
# Preparaing DataLoader

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

transform = transforms.Compose([transforms.Resize([224, 224]),
                                transforms.ToTensor()])
dataset = datasets.ImageFolder('/content/gdrive/My Drive/Colab Notebooks/Datasets/Living_image/', transform=transform)
dataloader = DataLoader(dataset, batch_size=296, shuffle=True)

In [ ]:
# Model creating

import torch.nn as nn, torch.optim as optim, torch.nn.functional as F, matplotlib.pyplot as plt
from torch import flatten, round

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3,32,3,1)
        self.conv2 = nn.Conv2d(32,64,3,1)
        self.fc1 = nn.Linear(186624, 256)       # the flatten data will be the multiply of final image size after conv and maxpool and the final filter output size
        self.fc2 = nn.Linear(256, 32)
        self.fc3 = nn.Linear(32, 4)
    def forward(self,x):
        x=self.conv1(x)
        x=F.leaky_relu(x)
        x=F.max_pool2d(x,2)
        x=self.conv2(x)
        x=F.leaky_relu(x)
        x=F.max_pool2d(x,2)
        x = flatten(x, 1)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

model = CNN()
model = model.to('cuda:0')
optimizer = optim.Adam(model.parameters(), lr=0.1, betas=(0.9, 0.999), eps=1e-08)

In [ ]:
# Training

epoch = 30
for n in range(epoch):
  print("\n")
  for (i,l) in dataloader:
    optimizer.zero_grad()
    i = i.to('cuda:0')
    l = l.to('cuda:0')
    bs, c, h, w = i.size()
    output = model(i.view(-1, c, h, w))
    loss = F.nll_loss(output, l)
    loss.backward()
    optimizer.step()
    print("Loss in {} No. epoch: {}".format(str(n+1), str(round(loss))), end='\r')

In [ ]:
# Testing

model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
  for data, target in dataloader:
    data = data.to('cuda:0')
    target = target.to('cuda:0')
    output = model(data)
    test_loss += F.nll_loss(output, target, 
                         reduction='sum').item() 
    pred = output.argmax(dim=1, keepdim=True)  
    correct += pred.eq(target.view_as(pred)).sum().item()
test_loss /= len(dataloader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(dataloader.dataset),
    100. * correct / len(dataloader.dataset)))